# Introduction
In this write up, we will be describing our work for our capstone. A lot of the work done was done in the background, so the main machine learning and feature engineering work has been done in this notebook. We are aiming to not prove that campaign funding and/or other features affect campaign results, but to show what affect they have on campaign outcomes. Since there has been much reasearch into the subect of predicting campaign results, there is a need for other reasearch as stated above.

# Data Sets 
There are two main datasets that we are working with. Here we will provide an overall explanation of each of them and finally explain our thought process of combining the two together to be used with our models and for future feature engineering. 
## Campaign Funding Data
This dataset has -- rows of funding data and -- unique organizations/individuals that donated for California local elections. This data was given to us by --, a company that partnered with Code for San Francisco to provide secure and relativley cleaned data to use in exploration and to build their platform off of. This dataset provides data from 20-- to the most recent election. One row would represent one transaction from an individual organization. It also shows what state they are from, who they are donating to, and also what election year they are making a certain donation. One organization, company, or individual can donate multiple times, so to get a total contribution for an individual, there is some aggregation needed.

## Candidate Campaign Outcomes

In [8]:
#from fuzzywuzzy import fuzz



import datetime as dt
import os
import sys

import numpy as np
import pandas as pd
from scipy import interp
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, confusion_matrix, roc_curve, average_precision_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../')
from utilities import sql_utils as su
from utilities import model_eval_utils as meu

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.max_columns = 999
import sqlalchemy as sqla

DB_URI = os.getenv('CD_DWH')
engine = create_engine(DB_URI)

## Data Processing - Data Set Engineering
A huge portion of our capstone was focused on producing usable data that we could engineer in the future. We were provided with two datasets where names were spelt very differently, for example, "Jerry Brown" and "Gerald Brown Jr.". The query shown below is the result of name mapping scripts. We created a table close to a hashing table where each name is connected to an ID signifying that two names belonged to the same person. So "Jerry Brown" and "Gerals Brown Jr." would theoretically be connected to the same ID. However, these connections are not 100% accurate, and have required some human intervention to manually connect these names together, because some candidates have many names that are might be different from another name. 

In [6]:
# Load data
QUERY = """
select
   distinct( candidate_name) as dbname
from data_ingest.casos__california_candidate_statewide_election_results

union

select
   distinct( recipient_candidate_name)
from trg_analytics.candidate_contributions
"""
with engine.begin() as conn:
    sql_results = pd.read_sql(QUERY, conn)

#sql_results = sql_results.dropna()
print("Size:", len(sql_results))
sql_results.head(5)

Size: 2574


,dbname
0,None
1,"KNIGHT, WILLIAM J. \PETE\"""""
2,Adam Shbeita
3,"HILL, JERRY A."
4,"ACOSTA, GEORGIA L."
5,"GUILLEN, ABEL"
6,Mario A. Guerra
7,"STOKER, MIKE"
8,"PARRA, PETE H."
9,"RUNNER, GEORGE C."


In [7]:
# Load pairings
csv_data = pd.read_csv("./dbname_2_id___2.csv")
csv_data = csv_data.set_index("dbname") # Now we can do csv_data.loc[str(x)].id to get the id

def dbname_to_id(name):
    try:
        return int(csv_data.loc[str(name)].id)
    except:
        return None

csv_data.head(5)

FileNotFoundError: File b'./dbname_2_id___2.csv' does not exist